### Data loader

This function loads a few weeks of the dataset, keeps only the useful columns, stitches all weeks together, and attaches the play context to both input and output frames so we can immediately compute features and our five pillars.

In [1]:
import pandas as pd
from pathlib import Path

def load_weeks_pandas(weeks, base="../data"):
    use_in = ["game_id","play_id","nfl_id","frame_id",
              "player_name","player_position","player_role","player_side",
              "x","y","s","a","o","dir",
              "num_frames_output","ball_land_x","ball_land_y"]
    use_out = ["game_id","play_id","nfl_id","frame_id","x","y"]
    supp_cols = ["game_id","play_id","season","week","pass_result",
                 "team_coverage_man_zone","pass_length","route_of_targeted_receiver",
                 "yards_gained","expected_points","expected_points_added"]

    inputs  = [pd.read_csv(f"{base}/input_2023_w{w:02d}.csv", usecols=use_in)  for w in weeks]
    outputs = [pd.read_csv(f"{base}/output_2023_w{w:02d}.csv", usecols=use_out) for w in weeks]

    input_df  = pd.concat(inputs,  ignore_index=True)
    output_df = pd.concat(outputs, ignore_index=True)
    supp_df   = pd.read_csv(f"{base}/supplementary_data.csv", usecols=supp_cols)

    # join play context into input/output
    input_df  = input_df.merge(supp_df,  on=["game_id","play_id"], how="left")
    output_df = output_df.merge(supp_df, on=["game_id","play_id"], how="left")
    #Get only the first and last frames
    input_df = (
        input_df.sort_values(['play_id', 'nfl_id', 'frame_id'])
                .groupby(['play_id', 'nfl_id'], group_keys=False)
                .apply(lambda g: g.loc[g['frame_id'].isin([g['frame_id'].min(), g['frame_id'].max()])]).reset_index()
    )
    output_df = (
        output_df.sort_values(['play_id', 'nfl_id', 'frame_id'])
                .groupby(['play_id', 'nfl_id'], group_keys=False)
                .apply(lambda g: g.loc[g['frame_id'].isin([g['frame_id'].min(), g['frame_id'].max()])]).reset_index()
    )
    
    #filtering players so its only WR and CB
    input_df = input_df[(input_df.get('player_position') == 'WR') | (input_df.get('player_position') == 'CB')]

    output_df = output_df[output_df['nfl_id'].isin(input_df['nfl_id'])]

    #dropping unneccesary columns
    input_df = input_df.drop(columns=['index', 'player_name'])
    output_df = output_df.drop(columns=['index'])

    return input_df, output_df, supp_df

# Example:
# inp, out, supp = load_weeks_pandas([1,2,3])


In [2]:
inp, out, supp = load_weeks_pandas([1,2,3])

/var/folders/dg/hdz0102s321cbt41brs_8r040000gn/T/ipykernel_90998/3892070321.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.loc[g['frame_id'].isin([g['frame_id'].min(), g['frame_id'].max()])]).reset_index()
/var/folders/dg/hdz0102s321cbt41brs_8r040000gn/T/ipykernel_90998/3892070321.py:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.loc[g['frame_id'].isin([g['fram

<h2> Validating Joins

In [ ]:
def validate_join_and_integrity(input_df, output_df, supp_df):
    report = {}

    #Row counts
    report["input_rows"] = len(input_df)
    report["output_rows"] = len(output_df)
    report["supp_rows"] = len(supp_df)

    #Column consistency
    report["input_columns"] = list(input_df.columns)
    report["output_columns"] = list(output_df.columns)

    #Missing values
    input_nulls = input_df.isna().mean().round(3)
    output_nulls = output_df.isna().mean().round(3)
    report["top_nulls_input"] = input_nulls[input_nulls > 0].sort_values(ascending=False).head(10).to_dict()
    report["top_nulls_output"] = output_nulls[output_nulls > 0].sort_values(ascending=False).head(10).to_dict()

    #Key uniqueness checks
    key_input = input_df[['game_id','play_id','nfl_id','frame_id']]
    key_output = output_df[['game_id','play_id','nfl_id','frame_id']]
    report["duplicate_keys_input"] = key_input.duplicated().sum()
    report["duplicate_keys_output"] = key_output.duplicated().sum()

    #Referential integrity (plays in input/output exist in supp)
    input_play_match = input_df['play_id'].isin(supp_df['play_id']).mean()
    output_play_match = output_df['play_id'].isin(supp_df['play_id']).mean()
    report["input_play_match_%"] = round(100 * input_play_match, 2)
    report["output_play_match_%"] = round(100 * output_play_match, 2)

    #WR/CB only check
    valid_positions = {"WR", "CB"}
    invalid_positions = set(input_df['player_position'].unique()) - valid_positions
    report["invalid_positions_in_input"] = list(invalid_positions)

    #NFL IDs overlap check (input vs output)
    overlap_ratio = input_df['nfl_id'].isin(output_df['nfl_id']).mean()
    report["nfl_id_overlap_%"] = round(100 * overlap_ratio, 2)

    #Frame sanity check
    min_frame = input_df['frame_id'].min()
    max_frame = input_df['frame_id'].max()
    report["frame_range_input"] = (int(min_frame), int(max_frame))

    #Coordinate sanity check
    report["x_range_input"] = (input_df['x'].min(), input_df['x'].max())
    report["y_range_input"] = (input_df['y'].min(), input_df['y'].max())

    #Explosion detection (duplicate plays/frames)
    play_counts = input_df.groupby(['game_id','play_id']).size().describe().to_dict()
    report["frames_per_play_summary"] = {k: round(v,2) for k,v in play_counts.items()}

    #Summary message
    report["summary"] = (
        f"Validation completed.\n"
        f"Input rows: {report['input_rows']:,}, Output rows: {report['output_rows']:,}\n"
        f"Duplicate keys (input/output): {report['duplicate_keys_input']}/{report['duplicate_keys_output']}\n"
        f"Play match (input/output): {report['input_play_match_%']}% / {report['output_play_match_%']}%\n"
        f"NFL ID overlap: {report['nfl_id_overlap_%']}%\n"
        f"Invalid positions found: {len(report['invalid_positions_in_input'])}\n"
    )

    return report


def print_validation_report(report):
    print("===== NFL Big Data Bowl Validation Report =====")
    print(report["summary"])
    print("\n-- Nulls (Input) --")
    print(report["top_nulls_input"])
    print("\n-- Nulls (Output) --")
    print(report["top_nulls_output"])
    print("\n-- Frame range (input):", report["frame_range_input"])
    print("-- X range:", report["x_range_input"])
    print("-- Y range:", report["y_range_input"])
    print("\n-- Frames per play summary --")
    print(report["frames_per_play_summary"])
    if report["invalid_positions_in_input"]:
        print("\n Invalid positions present:", report["invalid_positions_in_input"])
    print("=================================================")
    

#Running report shows us the data integrity status, which in this case is good! No duplicates,
#and data looks good.
report = validate_join_and_integrity(inp, out, supp)
print_validation_report(report)

===== NFL Big Data Bowl Validation Report =====
Validation completed.
Input rows: 27,221, Output rows: 7,968
Duplicate keys (input/output): 0/0
Play match (input/output): 100.0% / 100.0%
NFL ID overlap: 99.03%
Invalid positions found: 0


-- Nulls (Input) --
{}

-- Nulls (Output) --
{}

-- Frame range (input): (1, 83)
-- X range: (np.float64(1.21), np.float64(119.86))
-- Y range: (np.float64(0.8), np.float64(52.87))

-- Frames per play summary --
{'count': 2570.0, 'mean': 10.59, 'std': 1.95, 'min': 4.0, '25%': 10.0, '50%': 10.0, '75%': 12.0, 'max': 18.0}
